<a href="https://colab.research.google.com/github/ajaypydipala/fmml_labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [16]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [17]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-17-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [18]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [19]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [20]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [21]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [22]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [23]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [24]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [25]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [26]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [27]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [28]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [29]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# New section Increasing the number of iterations may help to some extent when dealing with a very small training dataset, but it's not a guaranteed solution, and it comes with its own set of limitations. When you have a very small dataset, the primary challenge is the lack of diversity and representativeness, which can lead to overfitting and limited generalization. Here are some considerations:

Overfitting: With a small training dataset, models are more prone to overfitting, where they memorize the training examples rather than learning meaningful patterns. Increasing iterations can exacerbate this problem, especially if the model is complex, as it may continue to fit the noise in the data.

Diminishing Returns: Increasing the number of iterations can provide marginal improvements in performance up to a certain point, but after that, the model may plateau in terms of learning from the limited data available. You might experience diminishing returns in terms of model performance.

Generalization Challenges: No matter how many iterations you use, a model can only generalize as well as the data it was trained on. If the training dataset is very small, the model's ability to generalize to new, unseen data will be limited.

Validation Set Size: Increasing iterations will not address issues related to a small validation dataset. A small validation dataset can result in unreliable estimates of the model's performance.

To address the challenges of working with a very small dataset, consider the following strategies:

Data Augmentation: If possible, use data augmentation techniques to artificially increase the size of your training dataset by creating variations of your existing data.

Transfer Learning: Utilize pre-trained models and fine-tune them on your small dataset. Transfer learning leverages knowledge learned from larger datasets to improve performance on smaller datasets.

Regularization: Apply regularization techniques like L1 and L2 regularization to mitigate overfitting, which can be more pronounced in small datasets.

Simpler Models: Consider using simpler, more interpretable models that are less likely to overfit.

Collect More Data: If feasible, expanding your dataset by collecting more data can be one of the most effective solutions for improving model performance.

While increasing the number of iterations can be part of the solution, it's important to address the fundamental limitations of a small dataset through techniques like data augmentation, regularization, and model selection to avoid overfitting and improve the model's generalization ability.






# New section The number of iterations or epochs in the context of training a machine learning model can significantly impact the estimate of the model's performance. Whether you get a better estimate with higher iterations depends on several factors and is not a straightforward yes or no answer. Here are some considerations:

Underfitting and Overfitting: The number of iterations can affect the trade-off between underfitting and overfitting. If you have too few iterations, the model might underfit, meaning it hasn't learned the underlying patterns in the data. If you have too many iterations, the model may overfit, meaning it starts fitting the noise in the data. Finding the right balance is crucial.

Convergence: In some cases, increasing the number of iterations can help the model converge to a better solution. Convergence means that the model's training process stabilizes, and it is no longer making significant improvements. If your model hasn't fully converged, increasing iterations might help it reach a more optimal solution.

Computational Resources: Training with a higher number of iterations can be computationally expensive. Depending on the hardware and time constraints, you may need to strike a balance between achieving better performance and resource usage.

Early Stopping: To avoid overfitting, many practitioners use techniques like early stopping. This means monitoring the model's performance on a validation set and stopping training once the performance starts to degrade. In such cases, more iterations might not lead to a better estimate but could harm it.

Hyperparameter Tuning: The number of iterations is often one of the hyperparameters to tune. Different problems and datasets may require different numbers of iterations for optimal performance. Hyperparameter tuning involves experimentation to find the best setting for your specific task.

In summary, the effect of the number of iterations on the estimate of a machine learning model's performance is highly context-dependent. It is not guaranteed that higher iterations will always result in a better estimate. The key is to strike the right balance between training long enough to allow the model to learn the underlying patterns in the data without overfitting. Experimentation, validation sets, and hyperparameter tuning are essential to determine the optimal number of iterations for your specific problem.






# New section Cross-validation, while providing more consistent and robust estimates of model performance, doesn't necessarily provide a more accurate estimate of the test accuracy. Test accuracy refers to how well a model performs on completely unseen data, typically data that has not been used during training or cross-validation. Cross-validation helps estimate how well a model generalizes to new data, but it's still an estimate and not a direct measure of test accuracy.

To obtain an accurate estimate of test accuracy, you should have a separate and independent test dataset that your model has never seen before. The purpose of this test dataset is to simulate how your model would perform in a real-world scenario when it encounters entirely new, unseen examples. Test accuracy is the most reliable measure of your model's performance in this context.

Cross-validation is a valuable tool for model selection, hyperparameter tuning, and assessing a model's generalization performance within the confines of your dataset. However, it does not replace the need for a dedicated test dataset. The results obtained from cross-validation are indicative of how well your model is expected to perform on new, unseen data, but the true test accuracy can only be determined by evaluating the model on a separate test dataset.

### `*New section*`Averaging the validation accuracy across multiple splits can indeed provide more consistent and reliable results when assessing the performance of a machine learning model, especially in cases where random factors can affect the results. This technique is commonly referred to as cross-validation, and it serves several important purposes:

Reducing Variance: By performing multiple splits of your dataset into training and validation sets, you obtain multiple accuracy scores. Averaging these scores helps reduce the influence of random variations that might occur with a single split. This, in turn, provides a more stable estimate of the model's generalization performance.

More Robust Evaluation: Cross-validation provides a more robust evaluation of your model's performance because it assesses its effectiveness across different subsets of the data. This can help identify potential issues with overfitting or underfitting that might be specific to one random split but not others.

Better Generalization Estimation: When you average validation accuracy across multiple splits, you obtain a more reliable estimate of how well your model will perform on unseen data. This is especially important when you want to assess your model's generalization to new, unseen data, which is the ultimate goal of most machine learning models.

It's important to choose an appropriate cross-validation strategy for your specific problem, such as k-fold cross-validation or stratified cross-validation, depending on the nature of your data and the goals of your analysis. Cross-validation can be computationally more expensive because it involves training and validating the model multiple times, but the benefits in terms of result consistency and robustness often outweigh the additional computational cost.